<div style="text-align: center;">
<h1>NFL Elo Rating System Explained</h1>
</div>

## 1. Overview

- Every team starts with a base Elo rating (e.g., 1500).
- After each game, Elo ratings are updated based on:
  - Who won/lost (or tied)
  - The expected outcome (based on pre-game Elo)
  - The margin of victory
  - Home field advantage
- Upsets and blowouts cause bigger Elo changes.
- The bigger the surprise or the margin, the bigger the Elo change.

<br></br>

---

## 2. Step-by-Step

We'll walk through the Elo update for a single game, with formulas and Python code for each step.

### 2.1. Expected Score Calculation

The expected score is the probability each team has to win, based on their Elo ratings and home field advantage.

<br></br>

$$
\text{expected\_home} = \frac{1}{1 + 10^{\frac{(\text{away\_elo} - (\text{home\_elo} + \text{hfa}))}{400}}}
$$

$$
\text{expected\_away} = \frac{1}{1 + 10^{\frac{((\text{home\_elo} + \text{hfa}) - \text{away\_elo})}{400}}}
$$


In [ ]:
# Example Elo ratings and home field advantage
home_elo = 1500
away_elo = 1500
hfa = 2.5  # Home field advantage

expected_home = 1 / (1 + 10 ** ((away_elo - (home_elo + hfa)) / 400))
expected_away = 1 / (1 + 10 ** (((home_elo + hfa) - away_elo) / 400))

print(f"Expected home win probability: {expected_home:.3f}")
print(f"Expected away win probability: {expected_away:.3f}")


### 2.2. Actual Result and Margin

- If home team wins: `actual_home = 1`, `actual_away = 0`
- If away team wins: `actual_home = 0`, `actual_away = 1`
- If tie: `actual_home = 0.5`, `actual_away = 0.5`

The margin of victory is used to scale the Elo change.


In [ ]:
# Example game result
home_score = 24
away_score = 17

if home_score > away_score:
    actual_home, actual_away = 1, 0
    score_diff = home_score - away_score
elif away_score > home_score:
    actual_home, actual_away = 0, 1
    score_diff = away_score - home_score
else:
    actual_home, actual_away = 0.5, 0.5
    score_diff = 0

print(f"Actual home: {actual_home}, Actual away: {actual_away}, Margin: {score_diff}")


### 2.3. K-Factor Adjustment (Margin of Victory)

The K-factor is scaled up for bigger wins:

<br></br>

$$
\text{k\_adj\_factor} = 1 + \frac{\log(|\text{score\_diff}| + 1)}{10}
$$

In [ ]:
import numpy as np
k_adj_factor = 1 + np.log(abs(score_diff) + 1) / 10
print(f"K-factor adjustment: {k_adj_factor:.3f}")

### 2.4. Elo Update Step

The Elo ratings are updated for both teams:

<br></br>

$$
\text{new\_elo} = \text{old\_elo} + K \times \text{k\_adj\_factor} \times (\text{actual} - \text{expected})
$$


In [ ]:
K = 20  # base K-factor
new_home_elo = home_elo + K * k_adj_factor * (actual_home - expected_home)
new_away_elo = away_elo + K * k_adj_factor * (actual_away - expected_away)

print(f"New home Elo: {new_home_elo:.2f}")
print(f"New away Elo: {new_away_elo:.2f}")
print(f"Home Elo change: {new_home_elo - home_elo:+.2f}")
print(f"Away Elo change: {new_away_elo - away_elo:+.2f}")


---

# Changing Parameters

This Elo formula is **very standard** and is the classic approach for rating systems in chess, sports, and more. However, **different models can produce different results** because of several key factors:

#### 1. **Parameter Choices**
- **K-factor:** Some models use a higher or lower K (how fast Elo changes). A higher K makes ratings more volatile.
- **Home Field Advantage (HFA):** The value for HFA can vary (e.g., 2.5, 3.0, or even 0).
- **Margin of Victory Adjustment:** Some models use a different formula or none at all for scaling Elo by the score difference.

#### 2. **Initialization**
- **Starting Elo:** Some models start all teams at 1500, others use historical performance or different baselines.

#### 3. **Game Selection**
- **Which games are included?** Some models use only regular season, others include playoffs, preseason, or even international games.
- **How far back do they go?** Some use only recent years, others use decades of data.

#### 4. **Custom Tweaks**
- Some models add extra adjustments for:
  - **Rest days**
  - **Travel distance**
  - **Injuries**
  - **Weather**
  - **Special weighting for recent games**
- Some use a **different conversion factor** (e.g., 400 in the denominator, or a different base for the exponent).

#### 5. **Post-Processing**
- Some models “regress” ratings toward the mean in the offseason.
- Some blend Elo with other power rankings or stats.

#### 6. **Randomness and Luck**
- Even with the same formula, Elo will diverge over time if you use different data, or if you reset at different points.

#### **Summary Table**

| Factor                        | Example Variation                | Effect on Results         |
|-------------------------------|----------------------------------|--------------------------|
| K-factor                      | 20 vs 30 vs 10                   | More/less rating movement|
| HFA                           | 2.5 vs 3.0 vs 0                  | Home teams more/less favored|
| Margin of Victory             | Log formula vs none vs linear    | Blowouts matter more/less|
| Data selection                | Only regular season vs all games | Different history        |
| Custom tweaks                 | Rest, injuries, etc.             | More/less realism        |
| Initialization                | 1500 vs custom                   | Early season differences |

#### **In summary:**
- The **core Elo formula is standard**, but **small changes in parameters, data, and tweaks** can lead to different results.
- That’s why you’ll see different Elo ratings and predictions from FiveThirtyEight, ESPN, betting sites, and your own model—even if they all use “Elo.”